# Compiling all data sources

In [9]:
import numpy as np
import pandas as pd
import requests
import json
import os
import time
opensecretsapi = os.getenv('opensecretsapi')
congressapi = os.getenv('congressapi')
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

## OpenSecrets: Members

In [2]:
states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [3]:
opensecrets_members = pd.DataFrame()
for s in states:
    #print(s)
    root = 'https://www.opensecrets.org/api/'
    headers = {'User-Agent': useragent,
              'From': 'jkropko@virginia.edu'}
    params = {'method': 'getLegislators',
              'id': s,
              'output': 'json',
              'apikey':opensecretsapi}
    r = requests.get(root, params=params, headers=headers)
    myjson = json.loads(r.text)
    leg = pd.json_normalize(myjson, record_path=['response', 'legislator'])
    opensecrets_members = pd.concat([opensecrets_members, leg])

In [4]:
opensecrets_members.columns = opensecrets_members.columns.str.replace('@attributes.', '')
opensecrets_members.to_csv('Data/opensecrets_members.csv', index=False)

## https://theunitedstates.io/: Members

In [5]:
url = 'https://theunitedstates.io/congress-legislators/legislators-current.json'
r = requests.get(url, headers=headers)
myjson = json.loads(r.text)
usio_members = pd.json_normalize(myjson)
usio_members.columns = usio_members.columns.str.replace('id.', '')
usio_members.to_csv('Data/usio_members.csv', index=False)

## Congress API: members

In [10]:
root = 'https://api.congress.gov'
params = {'api_key': congressapi}

bio_ids = usio_members['bioguide']
congress_members = pd.DataFrame()
i = 0
for b in bio_ids:
    time.sleep(5)
    if (i % 10)==0:
        print(f'Now working on member {i} of {len(bio_ids)}')
    if b != '':
        endpoint = f'/v3/member/{b}'
        r = requests.get(root + endpoint, params=params, headers=headers)
        myjson = json.loads(r.text)
        mem = pd.json_normalize(myjson['member'])
        congress_members = pd.concat([congress_members, mem])
    i += 1

Now working on member 0 of 539


ConnectionError: HTTPSConnectionPool(host='api.congress.gov', port=443): Max retries exceeded with url: /v3/member/C001070?api_key=6PQeGY3fU5pP9fbxRdG7tKwhgLgOoaG1SCdwqHTg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0xffff7ec13890>: Failed to resolve 'api.congress.gov' ([Errno -2] Name or service not known)"))

In [ ]:
congress_members.to_csv('congress_members.csv', index=False)

## Congress API: sponsored legislation

## Congress API: bills

## Congress API: committees/subcommittees

## https://theunitedstates.io/: committee assignments

## OpenSecrets API: members

## OpenSecrets API: contributions

## Voteview: voting similarity matrix

## Voteview: ideology

## MIT Elections: House and Senate